これはSaketryサイトから必要なデータセットを作成するためにスクレイピングするためのものです。

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 2. 情報を取得したいページのURL
url = 'https://www.saketry.com/113445.html'

print(f"これから、このページの情報を取得します: {url}")

# 3. WebページにアクセスしてHTML情報を取得する
try:
    response = requests.get(url)
    # サーバーからの応答がエラーだった場合に、ここでプログラムを停止させる
    response.raise_for_status() 
    print("ページのHTML情報を取得しました。")
except requests.exceptions.RequestException as e:
    print(f"エラー: ページにアクセスできませんでした。 {e}")
    exit() # プログラムを終了

# 4. BeautifulSoupを使って、HTMLを解析できる形に変換する
# 'html.parser'は、HTMLを解析するためのエンジン名
soup = BeautifulSoup(response.content, 'html.parser')
print("HTMLの解析準備ができました。")

# 5. ステップ2で調べた「住所」を元に、各情報を探し出す
print("各情報の抽出を開始します...")

# ▼ 商品名の取得
# 実際の住所である 'ty-product-block-title' を指定する
product_name_tag = soup.find('h1', class_='ty-product-block-title')

# .find('bdi') を追加する
# <h1>の中にある<bdi>タグをさらに見つけて、その中のテキストを取得する
product_name = product_name_tag.find('bdi').text.strip() if product_name_tag else '商品名不明'

print(f"  - 商品名: {product_name}")

# ▼ テイスティングノートの取得
description_area = soup.find('div', id='content_description')
tasting_note_parts = [] # 抽出したノートの各行を入れるための空リスト

if description_area:
    # description_areaの中から、全てのpタグを見つける
    paragraphs = description_area.find_all('p')
    
    for p in paragraphs:
        text = p.text.strip()
        # テキストが「香り：」「味わい：」「フィニッシュ：」のいずれかで始まるかチェック
        if text.startswith(('香り：', '味わい：', 'フィニッシュ：')):
            # 条件に一致した行だけをリストに追加する
            tasting_note_parts.append(text)
            
# 抽出した各行を、改行で連結して一つのテキストにまとめる
tasting_note = '\n'.join(tasting_note_parts)

print("  - テイスティングノートをピンポイントで取得しました。")


# ▼ スペック情報の取得
spec_data = {} # スペック情報を入れるための空の辞書を用意

# id='content_features'を持つdiv要素を見つける
spec_area = soup.find('div', id='content_features')

if spec_area:
    # spec_areaの中から、クラス名が'ty-product-feature'のdivを全て見つける
    spec_items = spec_area.find_all('div', class_='ty-product-feature')
    
    # 見つけた各項目をループ処理
    for item in spec_items:
        # ラベル（キー）と値（バリュー）をそれぞれ探す
        key_tag = item.find('span', class_='ty-product-feature__label')
        value_tag = item.find('div', class_='ty-product-feature__value')
        
        # キーと値の両方が見つかった場合のみ処理する
        if key_tag and value_tag:
            # .textで文字だけを取り出し、.strip()で余分な空白を除去
            # キーからコロン「:」も除去しておく
            key = key_tag.text.strip().replace(':', '') 
            value = value_tag.text.strip()
            
            # 辞書に保存する
            spec_data[key] = value
            print(f"  - {key}: {value}")
else:
    print("スペック情報が見つかりませんでした。")


# 6. 最終的にCSVに保存するために、全データを一つの辞書にまとめる
final_data = {
    '商品名': product_name,
    'テイスティングノート': tasting_note,
    # spec_data辞書を展開して結合
    **spec_data 
}
from pathlib import Path 
df = pd.DataFrame([final_data])

output_dir = Path('output')
output_dir.mkdir(parents=True, exist_ok=True)
file_path = output_dir / 'whisky_data_single.csv'
df.to_csv(file_path, index=False, encoding='utf-8-sig')


print("\n処理が完了しました！")
print(f"'{output_dir}'フォルダの中に '{file_path.name}' という名前でファイルが作成されています。")

これから、このページの情報を取得します: https://www.saketry.com/113445.html
ページのHTML情報を取得しました。
HTMLの解析準備ができました。
各情報の抽出を開始します...
  - 商品名: ベンリネス 2010 14年 リフィル シェリーカスク 58.2% 700mlザ・モルトマン
  - テイスティングノートをピンポイントで取得しました。
  - 原産国: スコットランド
  - 地域: スぺイサイド
  - ブランド: ザ・モルトマン
  - 銘柄: ベンリネス蒸溜所
  - メーカー: メドウサイド・ブレンディング
  - ビンテージ: 2010
  - 年数: 14
  - アルコール度数（%）: 58.2
  - 容量 (ml／g): 700

処理が完了しました！
'output'フォルダの中に 'whisky_data_single.csv' という名前でファイルが作成されています。


In [ ]:
# 1. 必要な道具をインポートする
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from pathlib import Path

# --- ここからがプログラムの本体 ---

# 1. URLリストが書かれたファイル名を指定
url_file = 'data\saketry_url_data.txt'

# 2. 【ループ前】抽出した全データを格納するための空のリストを用意
all_whisky_data = []

# 3. URLリストを読み込む
try:
    with open(url_file, 'r') as f:
        # strip()で各行の余分な空白や改行を削除
        urls = [line.strip() for line in f if line.strip()]
    print(f"--- {len(urls)}件のURLを読み込みました。スクレイピングを開始します。 ---")
except FileNotFoundError:
    print(f"エラー: {url_file} が見つかりません。処理を中断します。")
    exit()

# 4. 【ループ中】各URLを順番に処理
for i, url in enumerate(urls):
    print(f"\n--- {i+1}/{len(urls)}件目を処理中: {url} ---")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ▼ 各情報の抽出（ここは前回完成させたものと同じ）
        product_name_tag = soup.find('h1', class_='ty-product-block-title')
        product_name = product_name_tag.find('bdi').text.strip() if product_name_tag else '商品名不明'

        description_area = soup.find('div', id='content_description')
        tasting_note_parts = []
        if description_area:
            paragraphs = description_area.find_all('p')
            for p in paragraphs:
                text = p.text.strip()
                if text.startswith(('香り：', '味わい：', 'フィニッシュ：', 'アロマ：', 'フレーバー：')):
                 tasting_note_parts.append(text)
        tasting_note = '\n'.join(tasting_note_parts)

        spec_data = {}
        spec_area = soup.find('div', id='content_features')
        if spec_area:
            spec_items = spec_area.find_all('div', class_='ty-product-feature')
            for item in spec_items:
                key_tag = item.find('span', class_='ty-product-feature__label')
                value_tag = item.find('div', class_='ty-product-feature__value')
                if key_tag and value_tag:
                    key = key_tag.text.strip().replace(':', '')
                    value = value_tag.text.strip()
                    spec_data[key] = value
        
        # １ページ分の全データを一つの辞書にまとめる
        single_whisky_data = {
            '商品名': product_name,
            'テイスティングノート': tasting_note,
            **spec_data
        }

        # 抽出した１ページ分のデータを、大きなリストに追加する
        all_whisky_data.append(single_whisky_data)
        print(f"「{product_name}」のデータを正常に取得しました。")

    except Exception as e:
        print(f"エラーが発生したため、このURLの処理をスキップします: {e}")
    
    # サーバーに配慮して待機
    wait_time = random.uniform(2, 5)
    print(f"{wait_time:.2f}秒待機します...")
    time.sleep(wait_time)

# 5. 【ループ後】全データが溜まったリストを、一括でCSVファイルに保存
if all_whisky_data:
    df = pd.DataFrame(all_whisky_data)
    
    # 保存先フォルダとファイルパスを指定
    output_dir = Path('output')
    output_dir.mkdir(parents=True, exist_ok=True)
    file_path = output_dir / 'whisky_dataset_final.csv'
    
    # CSVファイルとして出力
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    
    print(f"\n--- 全ての処理が完了しました！ ---")
    print(f"{len(all_whisky_data)}件のデータを抽出し、'{file_path}'に保存しました。")
else:
    print("\nデータを一件も抽出できませんでした。")

<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
C:\Users\fugat\AppData\Local\Temp\ipykernel_44124\1223795352.py:12: SyntaxWarning: invalid escape sequence '\s'
  url_file = 'data\saketry_url_data.txt'


--- 98件のURLを読み込みました。スクレイピングを開始します。 ---

--- 1/98件目を処理中: https://www.saketry.com/113472.html ---
「リンクウッド 2009 13年 バーボンカスク 51.3％ 700mlザ・テイスター」のデータを正常に取得しました。
2.11秒待機します...

--- 2/98件目を処理中: https://www.saketry.com/113444.html ---
「アードベッグ 2007 16年 スコッチモルト販売45周年記念ボトル 54.1% 700mlザ・モルトマン」のデータを正常に取得しました。
4.34秒待機します...

--- 3/98件目を処理中: http://saketry.com/113445.html ---
「ベンリネス 2010 14年 リフィル シェリーカスク 58.2% 700mlザ・モルトマン」のデータを正常に取得しました。
4.53秒待機します...

--- 4/98件目を処理中: https://www.saketry.com/113443.html ---
「コッツウォルズ  ハーベスト・シリーズ No.3 アンバーメドウ シングルモルトウイスキー 51.6% 700mlコッツウォルズ」のデータを正常に取得しました。
3.28秒待機します...

--- 5/98件目を処理中: https://www.saketry.com/113408.html ---
「成田一徹「THE DUFFTOWN」ラベル／レダイグ 2009 14年 ホグスヘッド 50.5% 700ml成田一徹 ラベル・ウイスキー」のデータを正常に取得しました。
4.59秒待機します...

--- 6/98件目を処理中: https://www.saketry.com/113302.html ---
「イクイノックス＆ソルスティス スプリングエディション 2024 ティーニニック 12年 シングルリフィル・ホグスヘッド 48.5% 700mlイクイノックス＆ソルスティス」のデータを正常に取得しました。
2.63秒待機します...

--- 7/98件目を処理中: https://www.saketry.com/113407.html ---
「『旅みやげ第三集 別府の朝』ラベ

In [2]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin

# --- 設定項目 ---
# スクレイピングを開始する最初のページ（ウイスキーカテゴリの1ページ目）
start_url = 'https://www.saketry.com/all/whisky/' 
output_file = 'whisky_urls.txt'

# --- プログラム本体 ---
collected_urls = set()
current_url = start_url 

print("--- URL収集を開始します ---")

while current_url:
    print(f"--- ページを処理中: {current_url} ---")
    
    try:
        response = requests.get(current_url, timeout=10)
        if response.status_code == 404:
            print("ページが見つかりませんでした (404)。最後のページに到達したと判断します。")
            break
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 各商品を囲んでいるコンテナを見つける
        product_containers = soup.find_all('div', class_=lambda c: c and c.startswith('ty-column'))
        
        if not product_containers:
            print("このページには商品が見つかりませんでした。")
        else:
            # 各コンテナから商品ページへのリンク(aタグ)を見つけ、URLを抽出する
            for container in product_containers:
                # ★★★ ここがあなたが見つけてくれた正しいクラス名 ★★★
                a_tag = container.find('a', class_='product-title')
                
                if a_tag and a_tag.has_attr('href'):
                    absolute_url = a_tag['href']
                    collected_urls.add(absolute_url)

            print(f"{len(product_containers)}件の商品リンクを発見。現在合計{len(collected_urls)}件。")

        # 『次のページ』ボタンを探す
        next_page_tag = soup.find('a', class_='ty-pagination__next')
        
        if next_page_tag and next_page_tag.has_attr('href'):
            current_url = next_page_tag['href']
        else:
            print("『次のページ』ボタンが見つかりませんでした。全ページの処理を完了します。")
            current_url = None
            
        time.sleep(2)

    except requests.exceptions.RequestException as e:
        print(f"エラーが発生したため、処理を中断します: {e}")
        break
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")
        break

# --- 収集した全URLをファイルに書き出す ---
if collected_urls:
    print(f"\n--- 合計{len(collected_urls)}件のユニークなURLを収集しました ---")
    with open(output_file, 'w', encoding='utf-8') as f:
        for url in sorted(list(collected_urls)):
            f.write(url + '\n')
    print(f"'{output_file}'に全URLを保存しました。")
else:
    print("URLを一件も収集できませんでした。")

--- URL収集を開始します ---
--- ページを処理中: https://www.saketry.com/all/whisky/ ---
32件の商品リンクを発見。現在合計32件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-2/ ---
32件の商品リンクを発見。現在合計64件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-3/ ---
32件の商品リンクを発見。現在合計96件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-4/ ---
32件の商品リンクを発見。現在合計128件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-5/ ---
32件の商品リンクを発見。現在合計160件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-6/ ---
32件の商品リンクを発見。現在合計192件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-7/ ---
32件の商品リンクを発見。現在合計224件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-8/ ---
32件の商品リンクを発見。現在合計256件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-9/ ---
32件の商品リンクを発見。現在合計288件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-10/ ---
32件の商品リンクを発見。現在合計320件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-11/ ---
32件の商品リンクを発見。現在合計352件。
--- ページを処理中: https://www.saketry.com/all/whisky/page-12/ ---
32件の商品リンクを発見。現

In [4]:
# 1. 必要な道具をインポートする
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from pathlib import Path

# --- プログラムの本体 ---

# 1. URLリストが書かれたファイル名を指定
url_file = 'whisky_urls.txt' 

# 2. URLリストを読み込む
try:
    with open(url_file, 'r', encoding='utf-8') as f:
        urls = [line.strip() for line in f if line.strip()]
    print(f"--- {len(urls)}件のURLを読み込みました。詳細情報の収集を開始します。 ---")
except FileNotFoundError:
    print(f"エラー: '{url_file}' が見つかりません。先に`url_collector.py`を実行してください。")
    exit()

# --- 保存処理の準備 ---
# 保存先フォルダとファイルパスを指定
output_dir = Path('output')
output_dir.mkdir(parents=True, exist_ok=True)
file_path = output_dir / 'whisky_dataset_final.csv'

# 3. 【ループ中】各URLを順番に処理
for i, url in enumerate(urls):
    print(f"\n--- {i+1}/{len(urls)}件目を処理中: {url} ---")
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ▼ 各情報の抽出ロジック
        product_name_tag = soup.find('h1', class_='ty-product-block-title')
        product_name = product_name_tag.find('bdi').text.strip() if product_name_tag else '商品名不明'

        description_area = soup.find('div', id='content_description')
        tasting_note_parts = []
        if description_area:
            paragraphs = description_area.find_all('p')
            for p in paragraphs:
                text = p.text.strip()
                if text.startswith(('香り：', '味わい：', 'フィニッシュ：', 'アロマ：', 'フレーバー：')):
                    tasting_note_parts.append(text)
        tasting_note = '\n'.join(tasting_note_parts)

        spec_data = {}
        spec_area = soup.find('div', id='content_features')
        if spec_area:
            spec_items = spec_area.find_all('div', class_='ty-product-feature')
            for item in spec_items:
                key_tag = item.find('span', class_='ty-product-feature__label')
                value_tag = item.find('div', class_='ty-product-feature__value')
                if key_tag and value_tag:
                    key = key_tag.text.strip().replace(':', '')
                    value = value_tag.text.strip()
                    spec_data[key] = value
        
        # １ページ分の全データを一つの辞書にまとめる
        single_whisky_data = {
            '商品名': product_name,
            'テイスティングノート': tasting_note,
            'URL': url,
            **spec_data
        }

        # --- ▼▼▼ ここがメモリに優しい保存ロジック ▼▼▼ ---
        # 1件分のデータをDataFrameに変換
        df_single = pd.DataFrame([single_whisky_data])

        # ループの初回(i=0)だけヘッダー付きで新規作成(mode='w')し、
        # 2回目以降はヘッダーなしで追記(mode='a')していく
        if i == 0:
            # ファイルを新規作成モード('w')で開く
            df_single.to_csv(file_path, mode='w', header=True, index=False, encoding='utf-8-sig')
            print(f"○ 「{product_name}」のデータを取得し、ファイルを新規作成しました。")
        else:
            # ファイルを追記モード('a')で開く
            df_single.to_csv(file_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            print(f"○ 「{product_name}」のデータを取得し、ファイルに追記しました。")

    except Exception as e:
        print(f"× エラーが発生したため、このURLの処理をスキップします: {e}")
    
    # サーバーに配慮して待機
    wait_time = random.uniform(2, 4)
    print(f"   {wait_time:.2f}秒待機します...")
    time.sleep(wait_time)

print(f"\n--- 全ての処理が完了しました！ ---")
print(f"'{file_path}'にデータが保存されています。")

--- 1076件のURLを読み込みました。詳細情報の収集を開始します。 ---

--- 1/1076件目を処理中: https://www.saketry.com/103120.html ---
○ 「クラシック オブ アイラ (10L樽)」のデータを取得し、ファイルを新規作成しました。
   2.40秒待機します...

--- 2/1076件目を処理中: https://www.saketry.com/103237.html ---
○ 「ハイランド・クラシック (10L樽)」のデータを取得し、ファイルに追記しました。
   2.06秒待機します...

--- 3/1076件目を処理中: https://www.saketry.com/105095.html ---
○ 「クラシック オブ アイラ 58% 700mlスコッチモルトセールス」のデータを取得し、ファイルに追記しました。
   3.60秒待機します...

--- 4/1076件目を処理中: https://www.saketry.com/110066.html ---
○ 「タリバーディン 1991 19年 46％ザ・モルトマン」のデータを取得し、ファイルに追記しました。
   2.85秒待機します...

--- 5/1076件目を処理中: https://www.saketry.com/110371.html ---
○ 「燻酒 アイラ シングルモルト 50% 700ml燻酒」のデータを取得し、ファイルに追記しました。
   3.84秒待機します...

--- 6/1076件目を処理中: https://www.saketry.com/110408.html ---
○ 「ミズナラウッドリザーブ 46%イチローズモルト」のデータを取得し、ファイルに追記しました。
   2.06秒待機します...

--- 7/1076件目を処理中: https://www.saketry.com/110409.html ---
○ 「イチローズモルトワインウッドリザーブ 46% 700mlイチローズモルト」のデータを取得し、ファイルに追記しました。
   3.65秒待機します...

--- 8/1076件目を処理中: https://www.saketry.com/110484.html ---
○ 

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from pathlib import Path

# --- プログラムの本体 ---
# 1. URLリストが書かれたファイル名を指定
url_file = 'whisky_urls.txt' 

# 2. 【ループ前】抽出した全データを格納するための空のリストを用意
all_whisky_data = []

# 3. URLリストを読み込む
try:
    with open(url_file, 'r', encoding='utf-8') as f:
        urls = [line.strip() for line in f if line.strip()]
    print(f"--- {len(urls)}件のURLを読み込みました。詳細情報の収集を開始します。 ---")
except FileNotFoundError:
    print(f"エラー: '{url_file}' が見つかりません。先に`url_collector.py`を実行してください。")
    exit()

# 4. 【ループ中】各URLを順番に処理
for i, url in enumerate(urls):
    print(f"\n--- {i+1}/{len(urls)}件目を処理中: {url} ---")
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ▼ 各情報の抽出ロジック
        product_name_tag = soup.find('h1', class_='ty-product-block-title')
        product_name = product_name_tag.find('bdi').text.strip() if product_name_tag else '商品名不明'

        description_area = soup.find('div', id='content_description')
        tasting_note_parts = []
        if description_area:
            paragraphs = description_area.find_all('p')
            for p in paragraphs:
                text = p.text.strip()
                if text.startswith(('香り：', '味わい：', 'フィニッシュ：', 'アロマ：', 'フレーバー：')):
                    tasting_note_parts.append(text)
        tasting_note = '\n'.join(tasting_note_parts)

        spec_data = {}
        spec_area = soup.find('div', id='content_features')
        if spec_area:
            spec_items = spec_area.find_all('div', class_='ty-product-feature')
            for item in spec_items:
                key_tag = item.find('span', class_='ty-product-feature__label')
                value_tag = item.find('div', class_='ty-product-feature__value')
                if key_tag and value_tag:
                    key = key_tag.text.strip().replace(':', '')
                    value = value_tag.text.strip()
                    spec_data[key] = value
        
        # １ページ分の全データを一つの辞書にまとめる
        single_whisky_data = {
            '商品名': product_name,
            'テイスティングノート': tasting_note,
            'URL': url,
            **spec_data
        }
        
        # ★★★ データをメモリ上のリストに溜め込む ★★★
        all_whisky_data.append(single_whisky_data)
        print(f"○ 「{product_name}」のデータを正常に取得しました。")

    except Exception as e:
        print(f"× エラーが発生したため、このURLの処理をスキップします: {e}")
    
    wait_time = random.uniform(2, 4)
    print(f"   {wait_time:.2f}秒待機します...")
    time.sleep(wait_time)

# 5. 【ループ後】全データが溜まったリストを、一括でCSVファイルに保存
if all_whisky_data:
    # ★★★ ここでPandasがカラムを自動で揃えてくれる ★★★
    df = pd.DataFrame(all_whisky_data)
    
    output_dir = Path('output')
    output_dir.mkdir(parents=True, exist_ok=True)
    file_path = output_dir / 'whisky_dataset_final.csv'
    
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    
    print(f"\n--- 全ての処理が完了しました！ ---")
    print(f"{len(all_whisky_data)}件のデータを抽出し、'{file_path}'に保存しました。")
else:
    print("\nデータを一件も抽出できませんでした。")

--- 1076件のURLを読み込みました。詳細情報の収集を開始します。 ---

--- 1/1076件目を処理中: https://www.saketry.com/103120.html ---
○ 「クラシック オブ アイラ (10L樽)」のデータを正常に取得しました。
   3.83秒待機します...

--- 2/1076件目を処理中: https://www.saketry.com/103237.html ---
○ 「ハイランド・クラシック (10L樽)」のデータを正常に取得しました。
   2.34秒待機します...

--- 3/1076件目を処理中: https://www.saketry.com/105095.html ---
○ 「クラシック オブ アイラ 58% 700mlスコッチモルトセールス」のデータを正常に取得しました。
   2.27秒待機します...

--- 4/1076件目を処理中: https://www.saketry.com/110066.html ---
○ 「タリバーディン 1991 19年 46％ザ・モルトマン」のデータを正常に取得しました。
   2.85秒待機します...

--- 5/1076件目を処理中: https://www.saketry.com/110371.html ---
○ 「燻酒 アイラ シングルモルト 50% 700ml燻酒」のデータを正常に取得しました。
   2.08秒待機します...

--- 6/1076件目を処理中: https://www.saketry.com/110408.html ---
○ 「ミズナラウッドリザーブ 46%イチローズモルト」のデータを正常に取得しました。
   2.85秒待機します...

--- 7/1076件目を処理中: https://www.saketry.com/110409.html ---
○ 「イチローズモルトワインウッドリザーブ 46% 700mlイチローズモルト」のデータを正常に取得しました。
   2.34秒待機します...

--- 8/1076件目を処理中: https://www.saketry.com/110484.html ---
○ 「アラン ポートカスク・フィニッシュ 50%」のデータを正常に取得しました。
   3.

In [13]:
df = pd.read_csv('output\whisky_dataset_final.csv')
df.head(100)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\fugat\AppData\Local\Temp\ipykernel_47244\639702345.py:1: SyntaxWarning: invalid escape sequence '\w'
  df = pd.read_csv('output\whisky_dataset_final.csv')


,商品名,テイスティングノート,URL,原産国,地域,ブランド,メーカー,アルコール度数（%）,容量 (ml／g),銘柄,ビンテージ,年数
0,クラシック オブ アイラ (10L樽),NaN,https://www.saketry.com/103120.html,スコットランド,アイラ,エイジング・バレル,スコッチモルト・セールス,58,10000.0,NaN,NaN,NaN
1,ハイランド・クラシック (10L樽),NaN,https://www.saketry.com/103237.html,スコットランド,ハイランド,エイジング・バレル,スコッチモルト・セールス,NaN,10000.0,NaN,NaN,NaN
2,クラシック オブ アイラ 58% 700mlスコッチモルトセールス,NaN,https://www.saketry.com/105095.html,スコットランド,アイラ,NaN,スコッチモルト・セールス,58,700.0,NaN,NaN,NaN
3,タリバーディン 1991 19年 46％ザ・モルトマン,NaN,https://www.saketry.com/110066.html,スコットランド,ハイランド,ザ・モルトマン,メドウサイド・ブレンディング,46,700.0,タリバーディン蒸留所,1991,19
4,燻酒 アイラ シングルモルト 50% 700ml燻酒,NaN,https://www.saketry.com/110371.html,スコットランド,アイラ,燻酒,スコッチモルト・セールス,50,700.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,プルトニー 2006 13年 シェリーバット 53.6% 700mlハートブラザーズ ファイ...,NaN,https://www.saketry.com/111500.html,スコットランド,ハイランド,ハートブラザーズ ファイネストコレクション,ハート・ブラザーズ,53.6,700.0,プルトニー蒸溜所,2006,NaN
96,キングスバーンズ ドリーム・トゥ・ドラム シングルモルト 46％ 700mlウィームス,NaN,https://www.saketry.com/111502.html,スコットランド,ローランド,キングスバーンズ,Wemyss Malts,46,700.0,NaN,NaN,NaN
97,グレンアラヒー 1990 29年 オロロソシェリーパンチョン 58.2％グレンアラヒー,NaN,https://www.saketry.com/111504.html,スコットランド,スペイサイド,グレンアラヒー,グレンアラヒー,58.2,700.0,NaN,1990,NaN
98,アードベッグ 2005 13年 バッチ17 48.4%ブティックウイスキー,NaN,https://www.saketry.com/111505.html,スコットランド,アイラ,ブティックウイスキー,NaN,48.5,500.0,NaN,2005,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   商品名         1074 non-null   object 
 1   テイスティングノート  541 non-null    object 
 2   URL         1074 non-null   object 
 3   原産国         1061 non-null   object 
 4   地域          967 non-null    object 
 5   ブランド        972 non-null    object 
 6   メーカー        883 non-null    object 
 7   アルコール度数（%）  1052 non-null   object 
 8   容量 (ml／g)   1057 non-null   float64
 9   銘柄          787 non-null    object 
 10  ビンテージ       694 non-null    object 
 11  年数          797 non-null    object 
dtypes: float64(1), object(11)
memory usage: 100.8+ KB


In [15]:
df.describe

<bound method NDFrame.describe of                                                     商品名 テイスティングノート  \
0                                   クラシック オブ アイラ (10L樽)        NaN   
1                                    ハイランド・クラシック (10L樽)        NaN   
2                     クラシック オブ アイラ 58% 700mlスコッチモルトセールス        NaN   
3                           タリバーディン 1991 19年 46％ザ・モルトマン        NaN   
4                            燻酒 アイラ シングルモルト 50% 700ml燻酒        NaN   
...                                                 ...        ...   
1069         レベルイェル チェリー バーボンリキュール  35% 750mlREBEL YELL        NaN   
1070  Secret Speyside Distillery(シークレット スペイサイド ディスティ...        NaN   
1071                                ギャリソン･ブラザーズ　シングルバレル        NaN   
1072                                  ギャリソン･ブラザーズ　ストレート        NaN   
1073  THOMPSON BROS. カリラ 2010 10年 52.9% 700MLTHOMPSO...        NaN   

                                                    URL      原産国      地域  \
0                   https://www.saketry.com/10312

In [16]:
df[df['テイスティングノート'].notna()]

,商品名,テイスティングノート,URL,原産国,地域,ブランド,メーカー,アルコール度数（%）,容量 (ml／g),銘柄,ビンテージ,年数
7,アラン ポートカスク・フィニッシュ 50%,香り：ベイクドアップル、スパイス、ビターオレンジ。\n味わい：バニラ、シトラス、ドライフルー...,https://www.saketry.com/110484.html,スコットランド,アイランズ,NaN,Whisk-e Limited,50,700.0,NaN,NaN,NaN
8,ティーリング シングルモルト 46% 700mlティーリング,香り： メロン、イチジク、トフィーとレモン。\n味わい： ドライドフルーツ、シトラス、バニラ...,https://www.saketry.com/110503.html,アイルランド,NaN,ティーリング,ティーリングウイスキー社,46,700.0,ティーリング蒸留所,NaN,NaN
24,アラン ソーテルヌ・カスク・フィニッシュ 50%,香り： 蜂蜜、柑橘、アプリコット、メロン。,https://www.saketry.com/110890.html,スコットランド,アイランズ,ワインカスクシリーズ,NaN,50,700.0,アイル オブ アラン蒸溜所,NaN,NaN
31,ブレンデッド モルト スコッチ ウイスキー キメラ Cask ref:CH1-2016 46...,香り：甘いシェリーの味わい、バニラ、アプリコットと完熟のバナナ、ほんのりとタバコの香りとピー...,https://www.saketry.com/110995.html,スコットランド,NaN,NaN,ブラックアダー,46,700.0,NaN,NaN,NaN
32,ピートリーク エンバース EMB3 59%ブラックアダー,香り：スモーキーでピーティ。カリカリのベーコンのハチミツがけ。ハム、革。\n味わい：濃いハチ...,https://www.saketry.com/111007.html,スコットランド,アイラ,ピートリーク,ブラックアダー,59,700.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1059,コッツウォルズ ハーベスト・シリーズ No.3 アンバーメドウ シングルモルトウイスキー ...,香り：ストーンフルーツ、ラフランス、ハニードーナツ、メープルシロップ、ピンクペッパー\n味わ...,https://www.saketry.com/113443.html,イングランド,コッツウォルズ,コッツウォルズ,コッツウォルズ,51.6,700.0,コッツウォルズ蒸溜所,NaN,NaN
1061,ベンリネス 2010 14年 リフィル シェリーカスク 58.2% 700mlザ・モルトマン,香り：シトラスと焦がしたオーク。\n味わい：クリーミーで纏わりつくようなチョコレートにプラム...,https://www.saketry.com/113445.html,スコットランド,スぺイサイド,ザ・モルトマン,メドウサイド・ブレンディング,58.2,700.0,ベンリネス蒸溜所,2010,14
1062,コッツウォルズ シングルモルトウイスキー 母の日限定ラッピング 46% 200mlコッツウォルズ,香り： マジパンの香りと軽やかなフルーツ（桃とアプリコット）が重なる、蜂蜜とバタースコッチ\...,https://www.saketry.com/113452.html,イングランド,コッツウォルズ,NaN,コッツウォルズ,46,NaN,コッツウォルズ蒸溜所,NaN,NaN
1064,ザ・ヒーラック シャトー・ビアック シングルモルトウイスキー 50% 700mlアイル・オブ...,香り：ピートスモークと甘草と甘いワインが穏やかに混ざり合う。\n味わい：果実味はより軽くなり...,https://www.saketry.com/113468.html,スコットランド,アイランズ,アイル・オブ・ハリス,アイル・オブ・ハリス蒸溜所,50,700.0,アイル・オブ・ハリス蒸溜所,NaN,NaN


In [2]:
import pandas as pd
import MeCab
from pathlib import Path

# --- 1. 準備：データの読み込み ---
# 前回作成したCSVファイルを読み込みます
csv_file_path = Path('output') / 'whisky_dataset_final.csv' # 必要に応じてファイル名を変更してください

try:
    df = pd.read_csv(csv_file_path)
    # 念のため、この段階でもノートが空の行は除外しておきます
    df.dropna(subset=['テイスティングノート'], inplace=True)
    df = df[df['テイスティングノート'].str.strip() != '']
    print(f"'{csv_file_path}' を読み込み、{len(df)}件のデータを処理対象とします。")
except FileNotFoundError:
    print(f"エラー: '{csv_file_path}' が見つかりません。")
    exit()

# --- 2. MeCabの設定 ---
# MeCabのTagger（タグ付け器）オブジェクトを作成します
# "-Owakati" は「分かち書き」モードを指定するおまじないで、
# 文章をスペース区切りの単語の羅列に変換してくれます。
try:
    mecab = MeCab.Tagger("-Owakati")
    print("MeCabの準備ができました。")
except RuntimeError:
    print("エラー: MeCabの初期化に失敗しました。MeCabが正しくインストールされているか確認してください。")
    exit()

# --- 3. 形態素解析を行う関数を定義 ---
# テキストを一つ受け取り、分かち書きした結果を返す関数
def tokenize_text(text):
    # mecab.parse()でテキストを解析し、末尾の不要な改行などを.strip()で削除
    return mecab.parse(text).strip()

# --- 4. DataFrameの各行に形態素解析を適用 ---
# .apply()は、「テイスティングノート」列の各行に対して、
# 作成した`tokenize_text`関数を順番に実行し、
# その結果を新しい'tokens'という列に格納する、という非常に便利な命令です。
print("形態素解析を開始します...（データ量によって少し時間がかかります）")
df['tokens'] = df['テイスティングノート'].apply(tokenize_text)
print("形態素解析が完了しました。")


# --- 5. 結果の確認 ---
# 元の文章と、単語に分割された結果を並べて表示してみましょう
print("\n▼ 形態素解析の結果（先頭5行）")
print(df[['テイスティングノート', 'tokens']].head())


# --- 6. (任意) 結果を新しいCSVファイルとして保存 ---
# save_path = Path('output') / 'whisky_dataset_tokenized.csv'
# df.to_csv(save_path, index=False, encoding='utf-8-sig')
# print(f"\n形態素解析後のデータを '{save_path}' に保存しました。")

'output\whisky_dataset_final.csv' を読み込み、541件のデータを処理対象とします。
MeCabの準備ができました。
形態素解析を開始します...（データ量によって少し時間がかかります）
形態素解析が完了しました。

▼ 形態素解析の結果（先頭5行）
                                           テイスティングノート  \
7   香り：ベイクドアップル、スパイス、ビターオレンジ。\n味わい：バニラ、シトラス、ドライフルー...   
8   香り：　メロン、イチジク、トフィーとレモン。\n味わい：　ドライドフルーツ、シトラス、バニラ...   
24                              香り： 蜂蜜、柑橘、アプリコット、メロン。   
31  香り：甘いシェリーの味わい、バニラ、アプリコットと完熟のバナナ、ほんのりとタバコの香りとピー...   
32  香り：スモーキーでピーティ。カリカリのベーコンのハチミツがけ。ハム、革。\n味わい：濃いハチ...   

                                               tokens  
7   香り ： ベイクドアップル 、 スパイス 、 ビターオレンジ 。 味わい ： バニラ 、 シ...  
8   香り ： 　 メロン 、 イチジク 、 トフィー と レモン 。 味わい ： 　 ドライドフ...  
24                      香り ： 蜂蜜 、 柑橘 、 アプリコット 、 メロン 。  
31  香り ： 甘い シェリー の 味わい 、 バニラ 、 アプリコット と 完熟 の バナナ 、...  
32  香り ： スモーキー で ピーティ 。 カリカリ の ベーコン の ハチミツ がけ 。 ハム...  


In [10]:
# --- 7. 分割した単語リストから、不要な単語（ノイズ）を除去する ---

# ストップワードのリストを定義（これは自由にカスタマイズできます）
stop_words = ['する', 'いる', 'ある', 'これ', 'それ', 'あれ', '思う', '感じ', '的', '的',
              '香り', '味わい', 'フィニッシュ', 'アロマ', 'フレーバー', 'ノート', 'テイスティング','かすか','微か','ほのか','わずか','少し','少々','やや','かなり','非常に',
              'とても', 'すごく', 'めちゃくちゃ', 'すごい', '大変', 'すごく', 'とても', 'かなり', '非常に', '極めて', '特に', '特別', '一番', '最高',
]

# 単語をお掃除する関数を定義
def clean_tokens(text):
    # テキストをスペースで単語リストに分割
    words = text.split(' ')
    
    # フィルタリング後の単語を入れるための空リスト
    cleaned_words = []
    
    for word in words:
        # 品詞情報などを取得（今回は使わないが、より高度な処理も可能）
        # parts = mecab.parse(word).split('\t')
        
        # ▼ フィルタリングの条件 ▼
        # 1. ストップワードリストに含まれていない
        # 2. 1文字だけの単語ではない（「、」や「。」などの記号や助詞を除外しやすい）
        # 3. 数字ではない
        if word not in stop_words and len(word) > 1 and not word.isnumeric():
            cleaned_words.append(word)
            
    # 掃除後の単語リストを、再度スペース区切りの一つのテキストに戻す
    return ' '.join(cleaned_words)


# --- 8. 作成したお掃除関数を、'tokens'列に適用 ---
print("\n形態素解析後のトークンをクリーニングします...")
df['tokens_cleaned'] = df['tokens'].apply(clean_tokens)
print("クリーニングが完了しました。")


# --- 9. クリーニング前と後を比較して確認 ---
print("\n▼ クリーニング前後の比較（先頭5行）")
print(df[['tokens', 'tokens_cleaned']].head())


# --- 10. (任意) 最終的なデータをCSVに保存 ---
save_path = Path('output') / 'whisky_dataset_cleaned.csv'
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"\nクリーニング後のデータを '{save_path}' に保存しました。")


形態素解析後のトークンをクリーニングします...
クリーニングが完了しました。

▼ クリーニング前後の比較（先頭5行）
                                               tokens  \
7   香り ： ベイクドアップル 、 スパイス 、 ビターオレンジ 。 味わい ： バニラ 、 シ...   
8   香り ： 　 メロン 、 イチジク 、 トフィー と レモン 。 味わい ： 　 ドライドフ...   
24                      香り ： 蜂蜜 、 柑橘 、 アプリコット 、 メロン 。   
31  香り ： 甘い シェリー の 味わい 、 バニラ 、 アプリコット と 完熟 の バナナ 、...   
32  香り ： スモーキー で ピーティ 。 カリカリ の ベーコン の ハチミツ がけ 。 ハム...   

                                       tokens_cleaned  
7   ベイクドアップル スパイス ビターオレンジ バニラ シトラス ドライ フルーツ ナッツ チョ...  
8   メロン イチジク トフィー レモン ドライドフルーツ シトラス バニラ スパイス クローブ ...  
24                                   蜂蜜 柑橘 アプリコット メロン  
31  甘い シェリー バニラ アプリコット 完熟 バナナ ほんのり タバコ ピート これら カラメ...  
32  スモーキー ピーティ カリカリ ベーコン ハチミツ がけ ハム 濃い ハチミツ よう なめら...  

クリーニング後のデータを 'output\whisky_dataset_cleaned.csv' に保存しました。


In [ ]:
# ランダムサンプリング
display(df[['テイスティングノート', 'tokens_cleaned']].sample(20))

,テイスティングノート,tokens_cleaned
987,香り：透明感のあるピーティーさを、甘さとフルーティーさが包んでいる。徐々にピート焚きした麦芽...,透明 ピーティー フルーティー 包ん 徐々に ピート 焚き 麦芽 美味しい 桜餅 柏餅 感じ...
586,香り：リンゴのコンポート、トフィー、ココナッツ、ミルクティ。\n味わい：トフィー、キャラメル...,リンゴ コンポート トフィー ココナッツ ミルクティ トフィー キャラメル バニラ クリーム...
1004,香り：ジンジャーと緑茶をまぶしたバニラスポンジケーキ\n味わい：オールドバーボンが香るバニラ...,ジンジャー 緑茶 まぶし バニラスポンジケーキ オールドバーボン 香る バニラ トフィーポッ...
927,香り：サクラやハナミズキの花。リンゴの皮。ビスケットとクリーム。\n味わい：桜餅。パティスリ...,サクラ ハナミズキ リンゴ ビスケット クリーム 桜餅 パティスリー モル ティー スパイス...
839,香り：明るく、シトラスのようで、甘いヘザーのようなピートスモークとチョークのようなミネラルの...,明るく シトラス よう 甘い ヘザー よう ピートスモーク チョーク よう ミネラル 側面 ...
779,香り：焼き菓子のフロランタン、バニラ、黒ブドウ、ブラックチェリー。\n味わい：モルティな甘み...,焼き 菓子 フロランタン バニラ ブドウ ブラック チェリー モルティ 甘み ブラックカラン...
909,香り：甘いショートブレッド、バニラ、青リンゴ、レモンの皮。\n味わい：甘いビスケット、さくら...,甘い ショートブレッド バニラ リンゴ レモン 甘い ビスケット さくらんぼ グラッセ ホワ...
357,香り：バランスが非常によいシェリー樽ウイスキーの香り。ナツメヤシ、プルーンなどドライフルーツ...,バランス 非常 よい シェリー ウイスキー ナツメヤシ プルーン など ドライ フルーツ 高...
153,香り：ナツメヤシなどのドライフルーツ。ナッツ入りのチョコレート。最後にリンゴの皮。フレーバー...,ナツメヤシ など ドライ フルーツ ナッツ 入り チョコレート 最後 リンゴ ビターチョコ ...
432,香り：レーズンの香り、バタースコッチのこってりとしたアマロ、シナモンやクローブといった甘やか...,レーズン バター スコッチ こってり アマ シナモン クローブ といった スパイス レーズン...


In [30]:
import pandas as pd
import MeCab
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("--- プログラムを開始します ---")

csv_file_path = Path('output') / 'whisky_dataset_final.csv'

try:
    df = pd.read_csv(csv_file_path)
    df.dropna(subset=['テイスティングノート'], inplace=True)
    df = df[df['テイスティングノート'].str.strip() != '']
    df.reset_index(drop=True, inplace=True)
    print(f"'{csv_file_path}' を読み込み、{len(df)}件のデータを処理対象とします。")
except FileNotFoundError:
    print(f"エラー: '{csv_file_path}' が見つかりません。")
    exit()

#形態素分析
print("\n--- ステップ2: テキストの前処理を開始します ---")
try:
    mecab = MeCab.Tagger("-Owakati")
except RuntimeError as e:
    print(f"エラー: MeCabの初期化に失敗しました。インストールを確認してください。 {e}")
    exit()

stop_words = ['する', 'いる', 'ある', 'これ', 'それ', 'あれ', '思う', '感じ', '的',
              '香り', '味わい', 'フィニッシュ', 'アロマ', 'フレーバー', 'ノート', 'テイスティング']

def clean_and_tokenize(text):
    wakati_text = mecab.parse(text).strip()
    words = wakati_text.split(' ')
    cleaned_words = []
    for word in words:
        if word not in stop_words and len(word) > 1 and not word.isnumeric():
            cleaned_words.append(word)
    return ' '.join(cleaned_words)

df['tokens_cleaned'] = df['テイスティングノート'].apply(clean_and_tokenize)
print("テキストの前処理が完了しました。")


# 重みづけ
smoky_words = ['スモーキー', 'ピート', 'ピーティ', 'ヨード', '煙', '燻製', '潮', '薬品', '正露丸']
fruity_words = ['フルーティー', 'フルーツ', '果実', 'リンゴ', '柑橘', 'レモン', 'オレンジ', 
                'ベリー', 'レーズン', 'ピーチ', 'アプリコット', 'パイナップル']
sherry_words = ['シェリー', 'ドライフルーツ', 'レーズン', 'カカオ', 'チョコレート']

def add_weights(text):
    words = text.split(' ')
    boosted_text = text
    
    # スモーキー系
    if any(word in text for word in smoky_words):
        boosted_text += ' スモーキー' * 10
        
    # フルーティー系
    if any(word in text for word in fruity_words):
        boosted_text += ' フルーティー' * 10
        
    # シェリー系
    if any(word in text for word in sherry_words):
        boosted_text += ' シェリー' * 10
        
    return boosted_text

df['tokens_boosted'] = df['tokens_cleaned'].apply(add_weights)

print("キーワードの重みづけが完了しました。")
print("\n--- ステップ3: レコメンドエンジンを構築します ---")

# TF-IDFベクトル化
vectorizer = TfidfVectorizer(max_features=2000)
tfidf_matrix = vectorizer.fit_transform(df['tokens_boosted']) 
print(f"TF-IDF行列を作成しました。(形状: {tfidf_matrix.shape})")

# コサイン類似度計算
cosine_sim_matrix = cosine_similarity(tfidf_matrix)
print(f"類似度行列を作成しました。(形状: {cosine_sim_matrix.shape})")

# レコメンド関数を定義
indices = pd.Series(df.index, index=df['商品名']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim_matrix):
    try:
        idx = indices[title]
    except KeyError:
        return f"エラー: '{title}' というウイスキーはデータセットに存在しません。"
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    whisky_indices = [i[0] for i in sim_scores]
    
    return df.iloc[whisky_indices]

print("レコメンドエンジンの準備が完了しました！")


print("\n--- ステップ4: レコメンド実行例 ---")
target_whisky = input("レコメンドを受けたいウイスキーの名前を入力してください: ")

recommendations = get_recommendations(target_whisky)

print(f"\n▼「{target_whisky}」が好きなあなたへのおすすめ:")
if isinstance(recommendations, pd.DataFrame):
    print(recommendations[['商品名', 'URL']])
else:
    print(recommendations)

--- プログラムを開始します ---
'output\whisky_dataset_final.csv' を読み込み、541件のデータを処理対象とします。

--- ステップ2: テキストの前処理を開始します ---
テキストの前処理が完了しました。
キーワードの重みづけが完了しました。

--- ステップ3: レコメンドエンジンを構築します ---
TF-IDF行列を作成しました。(形状: (541, 1813))
類似度行列を作成しました。(形状: (541, 541))
レコメンドエンジンの準備が完了しました！

--- ステップ4: レコメンド実行例 ---

▼「ブルックラディ 13年 バッチ11 47.6%ブティックウイスキー」が好きなあなたへのおすすめ:
                                                   商品名  \
232  グレンスペイ 2010 11年 カルヴァドスフィニッシュ 55% 700mlザ・クーパーズ・...   
132  ウイスキーギャラリー オーヘントッシャン2000 20年 58.1% 700ml Whisk...   
2                             アラン ソーテルヌ・カスク・フィニッシュ 50%   
33                    グレンフィディック12年 スペシャルリザーブ 40％ 700ml   
205                   バスカ― アイリッシュウイスキーバスカ― アイリッシュウイスキー   

                                     URL  
232  https://www.saketry.com/112472.html  
132  https://www.saketry.com/112252.html  
2    https://www.saketry.com/110890.html  
33   https://www.saketry.com/111956.html  
205  https://www.saketry.com/112413.html  
